In [1]:
import numpy as np
import pickle
import gym
from gym.wrappers import Monitor
import tensorflow as tf
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'

In [2]:
env_id = 'HalfCheetah-v1'
normalize_obs = True
# 'HalfCheetah-v1'
# 'Hopper-v1'
# 'MountainCarContinuous-v0'
# 'Reacher-v1'
save_recording = False
path_actors = '/media/flowers/3C3C66F13C66A59C/data_save/ddpg_study_baseline/data/HalfCheetah-v1/run_DDPG_standard_Memory_1M/60/tf_save/'
meta_file = 'best_actor_step1922000_score5063.meta'

In [3]:
# initialize environment
try:
    env = gym.make(env_id)
except NotImplementedError:
    print(env_id + ' does not refer to a gym environment')

if save_recording:
    saving_vid = '/media/flowers/3C3C66F13C66A59C/data_save/gym_recording/ddpg_cheetah_drop/'+weight_file[:-5]
    env = Monitor(env,saving_vid,force=True)
# env.directory = '/media/flowers/3C3C66F13C66A59C/data_save/gym_recording/ddp_cheetah_drop'
# extract space
action_space = np.array([env.action_space.low, env.action_space.high]).T
observation_space = np.array([env.observation_space.low, env.observation_space.high]).T
if len(np.array(env.reward_range).shape)==1:
    reward_space = np.array(env.reward_range).reshape(1,-1)
else:
    eward_space = np.array(env.reward_range)

reward_dim = reward_space.shape[0]
observation_dim = observation_space.shape[0]
action_dim = action_space.shape[0]
state_size = observation_dim

In [6]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    saver = tf.train.import_meta_graph(path_actors+meta_file)
    print(path_actors)
    saver.restore(sess,tf.train.latest_checkpoint(path_actors))
    graph = tf.get_default_graph()
    obs0 = graph.get_tensor_by_name("obs0:0")
    actor_tf = graph.get_tensor_by_name("actor/Tanh:0")
    
    n_tests = 1
    score = np.zeros([n_tests])
    for i in range(n_tests):
        done = False
        observations = [env.reset()]
        actions = []
        rewards = []
        while not done:
            last_obs = observations[-1].squeeze()
            feed_dict = {obs0: [last_obs]}
            action = sess.run(actor_tf, feed_dict=feed_dict)
            actions.append(action)
            env.render()
            out = env.step(actions[-1])
            observations.append(out[0])
            rewards.append(out[1])
            done = out[2]    
        score[i] = sum(rewards)
    print(score.max())
    print(score.min())

/media/flowers/3C3C66F13C66A59C/data_save/ddpg_study_baseline/data/HalfCheetah-v1/run_DDPG_standard_Memory_1M/60/tf_save/
INFO:tensorflow:Restoring parameters from /media/flowers/3C3C66F13C66A59C/data_save/ddpg_study_baseline/data/HalfCheetah-v1/run_DDPG_standard_Memory_1M/60/tf_save/best_actor_step1922000_score5063
5234.524665577567
5234.524665577567


In [5]:
observation_dim

17

In [ ]:
# score = sum(rewards)
print(score.max())
print(score.min()